In [5]:
import pandas as pd
import json
import numpy as np

In [6]:
muertes = pd.read_excel('MUERTES-CANMAMA-M40-MUN-COL.xlsx')
muertes


,DPMP,DPNOM,ANIO,FALLECIDAS
0,EL ENCANTO,AMAZONAS,2009,0
1,EL ENCANTO,AMAZONAS,2010,0
2,EL ENCANTO,AMAZONAS,2011,0
3,EL ENCANTO,AMAZONAS,2012,0
4,EL ENCANTO,AMAZONAS,2013,0
...,...,...,...,...
16825,SANTA ROSALÍA,VICHADA,2019,0
16826,SANTA ROSALÍA,VICHADA,2020,0
16827,SANTA ROSALÍA,VICHADA,2021,0
16828,SANTA ROSALÍA,VICHADA,2022,0


In [7]:
muertes.groupby(['DPNOM'])['FALLECIDAS'].sum()

DPNOM
AMAZONAS                                                       18
ANTIOQUIA                                                    6102
ARAUCA                                                        169
ARCHIPIÉLAGO DE SAN ANDRÉS Y PROVIDENCIA Y SANTA CATALINA      56
ATLÁNTICO                                                    2928
BOLÍVAR                                                      1664
BOYACÁ                                                        869
CALDAS                                                       1065
CAQUETÁ                                                       240
CASANARE                                                      218
CAUCA                                                         807
CESAR                                                         757
CHOCÓ                                                         132
CUNDINAMARCA                                                 8956
CÓRDOBA                                                      1006
GUAI

In [8]:
poblacion = pd.read_excel('POB-M40-MUNCOL-2009-2023.xlsx')
#poblacion

In [9]:
poblacion_total = poblacion[['DPNOM', 'DPMP', 'MPIO', 'ANIO', 'TOTAL_MUJERES']]
poblacion_total

,DPNOM,DPMP,MPIO,ANIO,TOTAL_MUJERES
0,AMAZONAS,EL ENCANTO,91263,2009,205
1,AMAZONAS,EL ENCANTO,91263,2010,213
2,AMAZONAS,EL ENCANTO,91263,2011,224
3,AMAZONAS,EL ENCANTO,91263,2012,229
4,AMAZONAS,EL ENCANTO,91263,2013,233
...,...,...,...,...,...
16825,VICHADA,SANTA ROSALÍA,99624,2019,505
16826,VICHADA,SANTA ROSALÍA,99624,2020,547
16827,VICHADA,SANTA ROSALÍA,99624,2021,571
16828,VICHADA,SANTA ROSALÍA,99624,2022,602


In [10]:
def calcular_tasa_mortalidad(muertes, poblacion, año_inicio=2009, año_fin=2023, k=100000):
    """
    Calcula la tasa cruda de mortalidad por municipio y departamento para mujeres mayores de 40 años.

    Parámetros:
    - muertes: DataFrame con columnas ['DPNOM', 'DPMP', 'ANIO', 'FALLECIDAS']
    - poblacion: DataFrame con columnas ['DPNOM', 'DPMP', 'MPIO', 'ANIO', 'TOTAL_MUJERES']
    - año_inicio: Año inicial del periodo (int)
    - año_fin: Año final del periodo (int)
    - k: Factor de estandarización (int, default 100000)

    Retorna:
    - DataFrame con columnas ['MPIO', 'DPNOM', 'DPMP', 'FALLECIDAS', 'TOTAL_MUJERES', 'Tasa_Mortalidad']
    """
    # Filtrado por años
    muertes_filtrado = muertes[(muertes['ANIO'] >= año_inicio) & (muertes['ANIO'] <= año_fin)]
    pob_filtrada = poblacion[(poblacion['ANIO'] >= año_inicio) & (poblacion['ANIO'] <= año_fin)]

    # Agrupación de muertes por departamento y municipio
    muertes_agg = muertes_filtrado.groupby(['DPNOM', 'DPMP'], as_index=False)['FALLECIDAS'].sum()

    # Agrupación de población (incluyendo MPIO para conservarlo)
    pob_agg = pob_filtrada.groupby(['DPNOM', 'DPMP', 'MPIO'], as_index=False)['TOTAL_MUJERES'].sum()

    # Unión de muertes con población (se une por DPNOM y DPMP)
    tasa = pd.merge(muertes_agg, pob_agg, on=['DPNOM', 'DPMP'])

    # Cálculo de la tasa de mortalidad
    tasa['Tasa_Mortalidad'] = (tasa['FALLECIDAS'] / tasa['TOTAL_MUJERES']) * k

    # Reordenar columnas
    columnas = ['MPIO', 'DPNOM', 'DPMP', 'FALLECIDAS', 'TOTAL_MUJERES', 'Tasa_Mortalidad']
    tasa = tasa[columnas]

    return tasa


In [11]:
tasa_mortalidad  = calcular_tasa_mortalidad(muertes, poblacion_total, año_inicio=2019, año_fin=2023, k = 100000)

In [12]:
tasa_mortalidad


,MPIO,DPNOM,DPMP,FALLECIDAS,TOTAL_MUJERES,Tasa_Mortalidad
0,91263,AMAZONAS,EL ENCANTO,0,1210,0.000000
1,91405,AMAZONAS,LA CHORRERA,0,1409,0.000000
2,91407,AMAZONAS,LA PEDRERA,0,2093,0.000000
3,91430,AMAZONAS,LA VICTORIA,0,563,0.000000
4,91001,AMAZONAS,LETICIA,9,32908,27.348973
...,...,...,...,...,...,...
1117,97889,VAUPÉS,YAVARATÉ,0,409,0.000000
1118,99773,VICHADA,CUMARIBO,0,36271,0.000000
1119,99524,VICHADA,LA PRIMAVERA,0,6743,0.000000
1120,99001,VICHADA,PUERTO CARREÑO,5,13929,35.896331


In [13]:
tasa_mortalidad.query(" Tasa_Mortalidad > 0")

,MPIO,DPNOM,DPMP,FALLECIDAS,TOTAL_MUJERES,Tasa_Mortalidad
4,91001,AMAZONAS,LETICIA,9,32908,27.348973
11,5002,ANTIOQUIA,ABEJORRAL,3,22831,13.140029
15,5031,ANTIOQUIA,AMALFI,2,24000,8.333333
16,5034,ANTIOQUIA,ANDES,5,46004,10.868620
17,5036,ANTIOQUIA,ANGELÓPOLIS,3,6485,46.260601
...,...,...,...,...,...,...
1107,76863,VALLE DEL CAUCA,VERSALLES,1,8499,11.766090
1109,76890,VALLE DEL CAUCA,YOTOCO,2,17120,11.682243
1110,76892,VALLE DEL CAUCA,YUMBO,13,108878,11.939970
1111,76895,VALLE DEL CAUCA,ZARZAL,7,46626,15.013083


In [14]:
# Resultados: mayor, menor y promedio
mayor = tasa_mortalidad.loc[tasa_mortalidad['Tasa_Mortalidad'].idxmax()]
menor = tasa_mortalidad.loc[tasa_mortalidad['Tasa_Mortalidad'].idxmin()]
promedio = tasa_mortalidad['Tasa_Mortalidad'].mean()



In [15]:
print("\nMayor tasa:", f"{mayor['DPNOM']} - {mayor['DPMP']}", f"{mayor['Tasa_Mortalidad']:.2f}")
print("Menor tasa:", f"{menor['DPNOM']} - {menor['DPMP']}", f"{menor['Tasa_Mortalidad']:.2f}")
print("Promedio:", f"{promedio:.2f}")



Mayor tasa: BOYACÁ - RONDÓN 126.62
Menor tasa: AMAZONAS - EL ENCANTO 0.00
Promedio: 10.32


In [16]:
mdata_mun = pd.read_excel('metadata_mun_col.xlsx')
mdata_mun

,MPIO,DPMP,DPNOM,GEOMETRY
0,91405,LA CHORRERA,AMAZONAS,"POLYGON ((5000339.8632 1499049.6806000005, 501..."
1,91669,SANTANDER,AMAZONAS,"POLYGON ((5109398.9191 1528841.3461000007, 510..."
2,91460,MIRITÍ-PARANÁ,AMAZONAS,"POLYGON ((5161666.8311 1560407.6130999997, 516..."
3,91430,LA VICTORIA,AMAZONAS,"POLYGON ((5201993.3468 1568778.1232999992, 520..."
4,52560,POTOSÍ,NARIÑO,"POLYGON ((4492613.2148 1651194.4537000004, 449..."
...,...,...,...,...
1116,25377,LA CALERA,CUNDINAMARCA,"POLYGON ((4888416.9414 2091042.9704999998, 488..."
1117,11001,BOGOTA DC,CUNDINAMARCA,"POLYGON ((4888146.8336 2090743.1349, 4888149.8..."
1118,25214,COTA,CUNDINAMARCA,"POLYGON ((4878745.1637 2093914.8238999993, 487..."
1119,25175,CHÍA,CUNDINAMARCA,"POLYGON ((4888146.8336 2090743.1349, 4888145.8..."


In [17]:
# Supongamos que 'tasa_mortalidad' es el DataFrame generado por la función calcular_tasa_mortalidad
# Y 'mdata_mun' es el DataFrame del archivo de metadatos geográficos

tasa_mortalidad_geo = pd.merge(tasa_mortalidad, mdata_mun[['MPIO', 'GEOMETRY']], on='MPIO', how='left')
tasa_mortalidad_geo

,MPIO,DPNOM,DPMP,FALLECIDAS,TOTAL_MUJERES,Tasa_Mortalidad,GEOMETRY
0,91263,AMAZONAS,EL ENCANTO,0,1210,0.000000,"POLYGON ((5000382.5009 1391996.4739999995, 503..."
1,91405,AMAZONAS,LA CHORRERA,0,1409,0.000000,"POLYGON ((5000339.8632 1499049.6806000005, 501..."
2,91407,AMAZONAS,LA PEDRERA,0,2093,0.000000,"POLYGON ((5305430.1299 1510105.1340999994, 530..."
3,91430,AMAZONAS,LA VICTORIA,0,563,0.000000,"POLYGON ((5201993.3468 1568778.1232999992, 520..."
4,91001,AMAZONAS,LETICIA,9,32908,27.348973,"POLYGON ((5365098.4831 1225751.6786000002, 536..."
...,...,...,...,...,...,...,...
1117,97889,VAUPÉS,YAVARATÉ,0,409,0.000000,"POLYGON ((5329524.3429 1620719.4519999996, 532..."
1118,99773,VICHADA,CUMARIBO,0,36271,0.000000,"POLYGON ((5297522.899 2087432.4010000005, 5297..."
1119,99524,VICHADA,LA PRIMAVERA,0,6743,0.000000,"POLYGON ((5439298.7499 2243101.0524000004, 543..."
1120,99001,VICHADA,PUERTO CARREÑO,5,13929,35.896331,"POLYGON ((5574392.7105 2151197.4635000005, 557..."


In [18]:
with open('vecinos_municipales.json', 'r', encoding='utf-8') as f:
    vecinos_mun = json.load(f)

vecinos_mun

{'91405': ['18756', '86573', '91263', '91530', '91536', '91669'],
 '91669': ['18756', '91405', '91460', '91536'],
 '18756': ['18150',
  '18410',
  '18460',
  '18753',
  '18785',
  '18860',
  '86571',
  '86573',
  '91405',
  '91430',
  '91460',
  '91530',
  '91669',
  '95015',
  '95200',
  '97511'],
 '86573': ['18756', '86568', '86571', '91405', '91530'],
 '91263': ['91405', '91530', '91536'],
 '91536': ['91263', '91405', '91407', '91460', '91669', '91798'],
 '91530': ['18756', '86573', '91263', '91405'],
 '91460': ['18756', '91407', '91430', '91536', '91669', '97511'],
 '91430': ['18756', '91460', '97511'],
 '91407': ['91460', '91536', '91798', '97511', '97666'],
 '97511': ['18756',
  '91407',
  '91430',
  '91460',
  '95200',
  '97001',
  '97161',
  '97666'],
 '52560': ['52215', '52356', '86320'],
 '52215': ['52356', '52560', '52573', '86320'],
 '86320': ['52001',
  '52215',
  '52287',
  '52356',
  '52560',
  '52573',
  '86568',
  '86569',
  '86865',
  '86885'],
 '52356': ['52022',
  '

# MORAN INDEX

In [19]:
# Asegura que los códigos estén como string
tasa_mortalidad['MPIO'] = tasa_mortalidad['MPIO'].astype(str)

# Diccionario de tasas: {mpio: tasa}
tasas = dict(zip(tasa_mortalidad['MPIO'], tasa_mortalidad['Tasa_Mortalidad']))

# Media de la tasa
z_bar = np.mean(list(tasas.values()))


In [20]:
numerador = 0
for i in tasas:
    for j in vecinos_mun.get(i, []):
        if j in tasas:
            numerador += (tasas[i] - z_bar) * (tasas[j] - z_bar)


In [21]:
denominador = sum((zi - z_bar)**2 for zi in tasas.values())


In [22]:
S0 = sum(len([j for j in vecinos_mun.get(i, []) if j in tasas]) for i in tasas)


In [23]:
n = len(tasas)
I = (n / S0) * (numerador / denominador)
print("Índice de Moran I:", I)


Índice de Moran I: -0.012795026981602843


In [26]:
import pandas as pd
import numpy as np
import json



def calcular_moran_municipal(muertes, poblacion_total, año_inicio, año_fin, k=100000):
    # 1. Calcular la tasa de mortalidad
    tasa_mortalidad = calcular_tasa_mortalidad(muertes, poblacion_total, año_inicio=año_inicio, año_fin=año_fin, k=k)

    # 2. Leer geometría municipal
    mdata_mun = pd.read_excel('metadata_mun_col.xlsx')
    tasa_mortalidad_geo = pd.merge(tasa_mortalidad, mdata_mun[['MPIO', 'GEOMETRY']], on='MPIO', how='left')

    # 3. Cargar vecinos municipales
    with open('vecinos_municipales.json', 'r', encoding='utf-8') as f:
        vecinos_mun = json.load(f)

    # 4. Asegurar que MPIO esté en string
    tasa_mortalidad['MPIO'] = tasa_mortalidad['MPIO'].astype(str)

    # 5. Crear diccionario de tasas
    tasas = dict(zip(tasa_mortalidad['MPIO'], tasa_mortalidad['Tasa_Mortalidad']))

    # 6. Calcular media de tasa
    z_bar = np.mean(list(tasas.values()))

    # 7. Calcular el numerador del índice de Moran
    numerador = 0
    for i in tasas:
        for j in vecinos_mun.get(i, []):
            if j in tasas:
                numerador += (tasas[i] - z_bar) * (tasas[j] - z_bar)

    # 8. Calcular el denominador
    denominador = sum((zi - z_bar)**2 for zi in tasas.values())

    # 9. Calcular S0 y número de áreas
    S0 = sum(len([j for j in vecinos_mun.get(i, []) if j in tasas]) for i in tasas)
    n = len(tasas)

    # 10. Índice de Moran
    I = (n / S0) * (numerador / denominador)

    print(f"Índice de Moran I ({año_inicio} - {año_fin}):", I)
    


In [27]:
# Ejecutar para los 4 rangos
calcular_moran_municipal(muertes, poblacion_total, 2009, 2023)
calcular_moran_municipal(muertes, poblacion_total, 2009, 2013)
calcular_moran_municipal(muertes, poblacion_total, 2014, 2018)
calcular_moran_municipal(muertes, poblacion_total, 2019, 2023)


Índice de Moran I (2009 - 2023): 0.14309674177856324
Índice de Moran I (2009 - 2013): 0.0753830796969453
Índice de Moran I (2014 - 2018): 0.10499102416088696
Índice de Moran I (2019 - 2023): -0.012795026981602843


In [28]:
import pandas as pd
import numpy as np
import json
import random

def calcular_moran_con_pvalor(muertes, poblacion_total, año_inicio, año_fin, k=100000, permutaciones=999, semilla=42):
    # Calcular tasa de mortalidad
    tasa_mortalidad = calcular_tasa_mortalidad(muertes, poblacion_total, año_inicio=año_inicio, año_fin=año_fin, k=k)
    tasa_mortalidad['MPIO'] = tasa_mortalidad['MPIO'].astype(str)

    # Diccionario de tasas
    tasas = dict(zip(tasa_mortalidad['MPIO'], tasa_mortalidad['Tasa_Mortalidad']))

    # Leer vecinos
    with open('vecinos_municipales.json', 'r', encoding='utf-8') as f:
        vecinos_mun = json.load(f)

    # Tasa media
    z_bar = np.mean(list(tasas.values()))

    # Índice de Moran observado
    def calcular_I(tasas_dict):
        z_bar_local = np.mean(list(tasas_dict.values()))
        numerador = 0
        for i in tasas_dict:
            for j in vecinos_mun.get(i, []):
                if j in tasas_dict:
                    numerador += (tasas_dict[i] - z_bar_local) * (tasas_dict[j] - z_bar_local)
        denominador = sum((zi - z_bar_local) ** 2 for zi in tasas_dict.values())
        S0 = sum(len([j for j in vecinos_mun.get(i, []) if j in tasas_dict]) for i in tasas_dict)
        n = len(tasas_dict)
        return (n / S0) * (numerador / denominador)

    I_observado = calcular_I(tasas)

    # Permutaciones
    random.seed(semilla)
    valores_I = []
    tasas_lista = list(tasas.values())
    llaves = list(tasas.keys())
    for _ in range(permutaciones):
        tasas_perm = dict(zip(llaves, random.sample(tasas_lista, len(tasas_lista))))
        I_perm = calcular_I(tasas_perm)
        valores_I.append(I_perm)

    # p-valor empírico
    extremos = sum(1 for val in valores_I if abs(val) >= abs(I_observado))
    p_valor = (extremos + 1) / (permutaciones + 1)  # corrección por "1 más"

    print(f"Índice de Moran I ({año_inicio}-{año_fin}): {I_observado:.4f}")
    print(f"p-valor (permutaciones={permutaciones}): {p_valor:.4f}")

    return I_observado, p_valor


In [29]:
calcular_moran_con_pvalor(muertes, poblacion_total, 2009, 2023)
calcular_moran_con_pvalor(muertes, poblacion_total, 2009, 2013)
calcular_moran_con_pvalor(muertes, poblacion_total, 2014, 2018)
calcular_moran_con_pvalor(muertes, poblacion_total, 2019, 2023)


Índice de Moran I (2009-2023): 0.1431
p-valor (permutaciones=999): 0.0010
Índice de Moran I (2009-2013): 0.0754
p-valor (permutaciones=999): 0.0010
Índice de Moran I (2014-2018): 0.1050
p-valor (permutaciones=999): 0.0010
Índice de Moran I (2019-2023): -0.0128
p-valor (permutaciones=999): 0.5000


(np.float64(-0.012795026981602843), 0.5)